# **CNN using TensorFlow 2.0**

This notebook aims to learn how to use Convolutional Neural Networks (CNN) using TensorFlow 2.0.

Thus, we will use the MNIST dataset.

## 1) Imports and preliminaries

As we will use TensorBoard to visualize the training of the CNN, we have to import it.

In [1]:
%load_ext tensorboard.notebook

We remove the previous logs.

In [2]:
!rm -rf ./logs/ 

Imports : 
* TensorFlow 2.0 ( pip install tensorflow==2.0.0-alpha0 OR pip install tensorflow-gpu==2.0.0-alpha0 )
* Datetime ( pip install DateTime )
* Os ( already in Python's standard library )

In [3]:
import tensorflow as tf
import datetime
import os
from tensorflow.keras import datasets, layers, models

## 2) Building the CNN object

In [4]:
class CNN():
    # As you give images to a CNN, the size of the images, and the amount of their channels must be fixed.
    # When initializing, the model is not generated yet, and so is the TensorBoard callback.
    def __init__(self, img_width, img_height, channels):
        self.img_width = img_width
        self.image_height = img_height
        self.channels = channels
        self.model = None
        self.tb_callback = None
    
    # When generating the model, we give some parameters so that we can customize the CNN as we want
    # Yet, we can customize the dropout rate, and the depth of each convolutional layer
    def gen_model(self, dropout=0.2, depth=32):
        self.model = models.Sequential()
        
        self.model.add(layers.Conv2D(depth, (3, 3), activation='relu', input_shape=(28, 28, 1)))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Dropout(dropout))
        
        self.model.add(layers.Conv2D(depth*2, (3, 3), activation='relu'))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Dropout(dropout))
        
        self.model.add(layers.Conv2D(depth*2, (3, 3), activation='relu'))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(depth*2, activation='relu'))
        self.model.add(layers.Dense(10, activation='softmax'))
        
        self.model.summary()
        
        return self.model
    
    # To train the model using supervised learning, you have to give the model some data, and the labels of these data
    # You can also specify a different optimizer than Adam (which is like a better gradient descent), a different calculation
    # of the loss (measure of the error to minimize), if you want to use TensorBoard to visualize the training session, and the 
    # amont of epochs.
    def train(self,
              train_images, 
              train_labels,
              optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              tensorboard=True,
              epochs=10
              ):
        
        self.model.compile(optimizer=optimizer, loss=loss,  metrics=['accuracy'])
        
        if tensorboard:
            log_dir="logs" + os.sep + "fit" + os.sep + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            self.tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            self.model.fit(train_images, train_labels, epochs=epochs, callbacks=[self.tb_callback])
        else:
            self.model.fit(train_images, train_labels, epochs=epochs)
  
    # To evaluate the model, you have to give the model some data (and their labels) that he has NEVER SEEN during the training,
    # as we want it to learn a general meaning of the data, and not some bias specific to the training dataset.
    def evaluate(self,test_images, test_labels):
        test_loss, test_acc = self.model.evaluate(test_images, test_labels)
        print("Loss : \n {}".format(test_loss))
        print("Accuracy : \n {}".format(test_acc)) 

## 3) Loading the training and test datasets

To keep it simple, we will use the very classic MNIST dataset.

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

Reshaping the data so the shape is compatible with our CNN.

In [6]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

## 4) Generating and training the CNN

Initialization and generation of the CNN

In [7]:
cnn = CNN(28,28,1)
cnn.gen_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          3

Training the CNN using the MNIST training dataset.

In [8]:
cnn.train(train_images, train_labels)

Epoch 1/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3400 - accuracy: 0.9165
Epoch 2/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0825 - accuracy: 0.9743
Epoch 3/10
60000/60000 [==============================] - 5s 83us/sample - loss: 0.0677 - accuracy: 0.9793
Epoch 4/10
60000/60000 [==============================] - 5s 83us/sample - loss: 0.0605 - accuracy: 0.9815
Epoch 5/10
60000/60000 [==============================] - 5s 84us/sample - loss: 0.0530 - accuracy: 0.9834
Epoch 6/10
60000/60000 [==============================] - 5s 84us/sample - loss: 0.0457 - accuracy: 0.9859
Epoch 7/10
60000/60000 [==============================] - 5s 84us/sample - loss: 0.0430 - accuracy: 0.9873
Epoch 8/10
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0422 - accuracy: 0.9877
Epoch 9/10
60000/60000 [==============================] - 5s 84us/sample - loss: 0.0405 - accuracy: 0.9883
Epoch 10/10
60000/60000 [===========

Saving the training logs using TensorBoard.

In [9]:
%tensorboard --logdir logs/fit

Now, to use TensorBoard, open a terminal and write the following line : 
* tensorboard --logdir=./logs